# RBM for Collaborative Filtering

## Activation of visible layer


In [1]:
import tensorflow as tf
import numpy as np

from rbm.util.util import softmax, Σ

session = tf.InteractiveSession()

### Parameters

In [2]:
movies = 4
ratings = 3
users = 2

visible_size = movies*ratings
hidden_size = 3
visible_shape = [users, visible_size]

hidden_sample = tf.constant([[0., 0., 1.]], dtype=tf.float32).T

In [3]:
users_ratings = np.array((
    (1, 2, 2, 0), # User 1
    (0, 1, 2, 1)  # User 2
))

def assign_rating(value):
    response = [0]*ratings
    response[value] = 1.
    return response

# Hidden and visible values
data = tf.constant(
    np.array(list(map(assign_rating, users_ratings.reshape([-1])))).reshape(visible_shape),
    dtype=tf.float32
)

print('User ratings')
tf.reshape(data, [users, -1, ratings]).eval()

User ratings


array([[[0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.]]], dtype=float32)

In [4]:
# Params values
W = 0.01 * tf.random_normal([hidden_size, visible_size], dtype=tf.float32)
b_v = 0.01 * tf.random_normal([visible_size, 1], dtype=tf.float32)
b_h = 0.01 * tf.random_normal([hidden_size, 1], dtype=tf.float32)

### Probability

In [5]:
x_sigmoid = hidden_sample.T @ W + b_v.T

# Remove all the movie have not been rating
# In really, remove all the movie-rating probabilities
mask = tf.sign(data)
x_maked = x_sigmoid * mask

# Reshape for apply softmax correctly
x = tf.reshape(x_maked, [users, -1, ratings])

# Calc probability
probabilies = softmax(x)

probabilies = tf.reshape(probabilies, [users, -1])

### Sample

In [6]:
samples = tf.nn.relu(tf.sign(probabilies - tf.random_uniform(tf.shape(probabilies))))

print('The generate samples does not respect the restriction')
tf.reshape(samples, [users, -1, ratings]).eval()

The generate samples does not respect the restriction


array([[[0., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 1.]],

       [[1., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 1.]]], dtype=float32)

## A simple test with argmax

In [7]:
probabilies_calculated = probabilies.eval()

c = x_sigmoid.eval()

print(tf.reshape(c, [users, -1, ratings]).eval())
tf.argmax(tf.reshape(c, [users, -1, ratings]), axis=2).eval()

[[[ 2.4445081e-02 -1.7063871e-02 -9.4049480e-03]
  [-2.5904500e-03 -5.4858215e-03 -3.0523026e-04]]

 [[ 5.1682736e-03 -2.1227937e-02  8.8326400e-05]
  [ 1.1481711e-02  2.0785579e-03  1.7699871e-02]]]


array([[0, 2],
       [0, 2]])